If you find any spelling mistakes in the comments, dont be baffled by that, I just did'nt put the required amount of effort necessary to write the comments (PS: (maybe)Its proof that AI did not write my code )

In [ ]:
#Firstly I installed kaggle in order to access the kaggle dataset using API
!pip install -q kaggle

In [ ]:
# This code is to upload a local file (Here:- kaggle API key file) into colab
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shoubhitwandile","key":"e9a7c8ee1739b87ba1bd1c1a4ed9f68a"}'}

In [ ]:
# Then I created a kaggle folder
!mkdir ~/.kaggle


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# Then I copied the kaggle.json file to folder created
!cp kaggle.json ~/.kaggle/



In [ ]:
# This gives permission for the json to act
# I dont know much about this, it was in the youtube video recommended
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Now this uses a API dataset code in order to directly download the dataset in form of zip file
!kaggle datasets download -d borismarjanovic/price-volume-data-for-all-us-stocks-etfs


Dataset URL: https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
License(s): CC0-1.0
price-volume-data-for-all-us-stocks-etfs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# This unzips the zip file downloaded and then we get all the files in the dataset
!unzip price-volume-data-for-all-us-stocks-etfs.zip


Archive:  price-volume-data-for-all-us-stocks-etfs.zip
replace Data/ETFs/aadr.us.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd                                                       # pandas is used for data analysis
import os                                                                 # os is used to interact with the operating system

tickers = ['aapl', 'msft', 'amzn', 'tsla', 'googl']                       # Defining a python list named TICKERS
data_frames = []                                                          # defining a empty list to store datasets


for ticker in tickers:                                                    # Loop for loading .txt files
    file_path = f'/content/Stocks/{ticker}.us.txt'                        # for each ticker I construced a file path using f string


    df = pd.read_csv(file_path, sep=',', parse_dates=['Date'])            # Using a built in function to read files into a dataframe
                                                                          # file_path = path to the file
                                                                          # sep=',' = this tells pandas to use commas as column seprators
                                                                          # parse_dates parameter instructs pandas to convert column 'date'-
                                                                          #--into actual datatime objects instead of plain text

    df['Ticker'] = ticker                                                 # Here I added a column named ticker to store the current company name
    data_frames.append(df)                                                # Now at the end of each iteration, I will be adding the created
                                                                          # dataframe into the previously created list


#---------------------------------AT THIS POINT WE HAVE A LIST OF 5 DIFFERENT DATAFRAMES------------------------------------------------------



combined_df = pd.concat(data_frames)                                      # This combines all the 5 dataframes into a single large datframe


multi_index_df = combined_df.set_index(['Ticker', 'Date']).sort_index()   # replacing row numbers with a multiIndex(two-level index)
                                                                          # This is done to avoid confusion while traversing columns
                                                                          # And then I have sorted it wrt ticker and date

multi_index_df.head()                                                     # This displays the first five rows(just to check)(no real use)


Open     High      Low    Close    Volume  OpenInt
Ticker Date                                                             
aapl   1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030        0
       1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532        0
       1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199        0
       1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801        0
       1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062        0

In [ ]:
present_tickers = multi_index_df.index.get_level_values('Ticker').unique().tolist()  # TO CHECK THE TICKERS
print(present_tickers)


['aapl', 'amzn', 'googl', 'msft', 'tsla']


In [ ]:
import pandas as pd
missing_per_ticker = multi_index_df.isna().groupby('Ticker').sum()                 # df.isna()--creates a boolean dataframe,True is stored where data is missing
                                                                           # .groupby('Ticker')--groups data b ticker column, for analysing missing data by company
                                                                           # .sum()-- sums the boolean values to get total number of values missing

print("Missing values per Ticker:\n", missing_per_ticker)       # TO CHECK THE MISSING VALUES










Missing values per Ticker:
         Open  High  Low  Close  Volume  OpenInt
Ticker                                         
aapl       0     0    0      0       0        0
amzn       0     0    0      0       0        0
googl      0     0    0      0       0        0
msft       0     0    0      0       0        0
tsla       0     0    0      0       0        0


In [ ]:
# NOW I WILL FILL THE MISSING VALUES(which is not required here)


# Option A: Interpolation (good for numerical data like prices)
#df = multi_index_df.groupby('Ticker').apply(lambda group: group.interpolate())

# Option B: Forward fill (use if data is non-numeric or interpolation isn’t appropriate)
#df = df.groupby('Ticker').apply(lambda group: group.ffill())

# Optional: Backward fill if forward fill doesn't fill all values
#df = df.groupby('Ticker').apply(lambda group: group.bfill())

In [ ]:
import pandas as pd

#  RECheck for missing values per ticker        #USEFUL WHEN VALUES ARE actually missing
missing_values = multi_index_df.isnull().groupby('Ticker').sum()
print(" Missing values per ticker:")
print(missing_values)






 Missing values per ticker:
        Open  High  Low  Close  Volume  OpenInt  Daily Return  7-day MA  \
Ticker                                                                    
aapl       0     0    0      0       0        0             1         6   
amzn       0     0    0      0       0        0             1         6   
googl      0     0    0      0       0        0             1         6   
msft       0     0    0      0       0        0             1         6   
tsla       0     0    0      0       0        0             1         6   

        30-day MA  Rolling Volatility (30d)  
Ticker                                       
aapl           29                        30  
amzn           29                        30  
googl          29                        30  
msft           29                        30  
tsla           29                        30  


In [ ]:
print(multi_index_df.index.names)                                # for checking the index


['Ticker', 'Date']


In [ ]:
# Making sure 'Date' is in columns and not in the index(else modification Not possible)
if 'Date' not in multi_index_df.columns:
    multi_index_df = multi_index_df.reset_index()

# conversation into datetime object
multi_index_df['Date'] = pd.to_datetime(multi_index_df['Date'])
multi_index_df = multi_index_df.set_index(['Ticker', 'Date']).sort_index() # making a multiindex and sorting


Index(['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt'], dtype='object')


In [ ]:
print(multi_index_df.index.is_monotonic_increasing)                                # This checks if the  index values are sorted(not necessary)


True


In [ ]:
import pandas as pd


latest_date = multi_index_df.index.get_level_values('Date').max()           # to get the last date in the dataset


cutoff_date = latest_date - pd.DateOffset(years=10)             # This date is 10 years prior to the latest date


multi_index_df = multi_index_df[multi_index_df.index.get_level_values('Date') >= cutoff_date]       # selection of rows(dates) greater tahn the cuttoff date


In [ ]:
multi_index_df = multi_index_df.reset_index()  # bring 'Date' and 'Ticker' back as columns
multi_index_df['Date'] = pd.to_datetime(multi_index_df['Date'])
multi_index_df = multi_index_df.set_index(['Ticker', 'Date']).sort_index()

#this code is for correcting some errors,writeen by chatGPT nad no idea why I did the convertion again
#                              ;-;

In [ ]:
# Group by Ticker to apply transformations independently
multi_index_df['Daily Return'] = multi_index_df.groupby(level='Ticker')['Close'].pct_change() * 100     # df.groupby--Splits the data by stocks
                                                                                                        # pct_change--computes change bet consq closes
                                                                                                        # mulltiply by 100 to convert into percentage



multi_index_df['7-day MA'] = multi_index_df.groupby(level='Ticker')['Close'].transform(lambda x: x.rolling(window=7).mean())    # transform keeps shape
multi_index_df['30-day MA'] = multi_index_df.groupby(level='Ticker')['Close'].transform(lambda x: x.rolling(window=30).mean())

# Rolling Volatility (30-day Std Dev of Daily Return)
multi_index_df['Rolling Volatility (30d)'] = multi_index_df.groupby(level='Ticker')['Daily Return'].transform(lambda x: x.rolling(window=30).std())


In [ ]:
print(multi_index_df.head(15))  # For seeing the new columns


                     Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604        0   
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0   
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0   
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0   
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0   
       2007-11-19  21.262  21.540  20.758  20.996  321694225        0   
       2007-11-20  21.247  22.000  20.942  21.623  429930531        0   
       2007-11-21  21.195  22.072  21.086  21.572  339509885        0   
       2007-11-23  22.027  22.033  21.739  21.967  129891462        0   
       2007-11-26  22.268  22.700  22.072  22.097  363869639        0   
       2007-11-27  22.439  22.511  21.772  22.387  366923874        0   
       2007-11-28  22.669  23.128  22.456  23.080  

In [ ]:
# Group by Ticker and calculate mean daily return and then the hishest daily return
avg_returns = multi_index_df.groupby(level='Ticker')['Daily Return'].mean()
highest_avg_return = avg_returns.idxmax()

print(f"Stock with highest average return: {highest_avg_return}")
print(avg_returns.sort_values(ascending=False))


Stock with highest average return: tsla
Ticker
tsla     0.210083
amzn     0.135932
aapl     0.106375
googl    0.064454
msft     0.058938
Name: Daily Return, dtype: float64


In [ ]:
# Group by Ticker and MONth, calculate std deviation of Daily Returns
monthly_vol = multi_index_df.groupby([pd.Grouper(level='Ticker'), pd.Grouper(level='Date', freq='M')])['Daily Return'].std()
#sorting the volatitlity in descneding ore=der& heads(1) gets the max volatitlity
most_volatile = monthly_vol.sort_values(ascending=False).head(1)
# t get the DATE
stock, date = most_volatile.index[0]
#                                         for formating                                    volatility value in 2 decimals
print(f"Most volatile month: {stock} in {date.strftime('%B %Y')} with volatility {most_volatile.iloc[0]:.2f}%")


Most volatile month: tsla in June 2010 with volatility 28.84%


<ipython-input-146-a0237c3e57c9>:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_vol = multi_index_df.groupby([pd.Grouper(level='Ticker'), pd.Grouper(level='Date', freq='M')])['Daily Return'].std()
